In [1]:
import torch
from torch import nn
from torch.autograd import Variable
from torch import optim

In [2]:
# torch.randn((2,1))
# optim.SGD?
# nn.CrossEntropyLoss?
# torch.rand?

In [3]:
class RNN(nn.Module):

    # you can also accept arguments in your model constructor
    def __init__(self, X_size, H_size, Y_size):
        super(RNN, self).__init__()

        self.X_size = X_size
        self.H_size = H_size
        self.Y_size = Y_size
        self.z_size = self.X_size + self.H_size
        
        self.i2h = nn.Linear(self.z_size, self.H_size)
        self.i2o = nn.Linear(self.z_size, self.Y_size)

    def forward(self, x, hidden_prev):
        z = torch.squeeze(torch.cat((x, hidden_prev)))
        hidden = self.i2h(z)
        y = self.i2o(z)
        return y, hidden
    
    def init_hidden(self):
        return Variable(torch.zeros((self.H_size)))
        

In [4]:
X_size = 50
H_size = 20
Y_size = 10
TIMES = 10

In [5]:
rnn = RNN(X_size, H_size, Y_size)

In [6]:
hidden = rnn.init_hidden()
x = Variable(torch.randn((TIMES, X_size)), requires_grad=True)
y = Variable(torch.rand((TIMES, Y_size)))
y_hat = Variable(torch.randn((TIMES, Y_size)))

criterion = nn.MSELoss() # loss

optimizer = optim.SGD(rnn.parameters(), lr=0.001)


for epoch in range(100):
    # 前向传播
    rnn.zero_grad()
    for t in range(TIMES):
        y_hat[t], hidden = rnn(x[t], hidden)
#         print('y_hat.size:', y_hat.size())
    loss = criterion(y_hat, y)
    if epoch%50==0:
        print('epoch:{}, loss:{}'.format(epoch, loss.data.numpy()[0]))
    loss.backward(retain_graph=True)
    optimizer.step()

epoch:0, loss:0.692116379738
epoch:50, loss:0.595489561558
